In [1]:
import pandas as pd

In [2]:
dictionary = {
    'Guns': 
        ['bb gun', 'pellet gun', 'air pistol', 'bean-bag gun', 'gun'],
    'Blunt instruments':
        ['hammer', 'axe', 'ax', 'hatchet', 'crowbar', 'pole', 'rod', 'walking stick', 'stick', 'rock', 'baton', 'shovel',
        'metal object', 'baseball bat', 'bat', 'flagpole', 'metal pole', 'metal stick', 'blunt object', 'metal pipe', 
         'carjack', 'brick', 'garden tool', 'metal rake', 'mace', 'wrench', 'pipe'],
    'Sharp objects':
        ['knife', 'bayonet', 'razor', 'blade', 'machete', 'sword', 'chainsaw', 'chain saw' , 'sharp object', 'scissor',
         'scissors','chain saw', 'glass shard', 'samurai sword', 'lawn mower blade', 'box-cutter', 'straight edge razor',
        'beer bottle', 'bottle', 'sharp object','meat cleaver', 'box cutter'],
    'Piercing objects':
        ['spear', 'pick-axe', 'pick axe', 'pitchfork', 'cordless drill', 'nail gun', 'nailgun', 'pen',
        'crossbow', 'arrow and bow', 'screwdriver', 'ice pick', 'bow and arrow'],
    'Other unusual objects':
        ['oar', 'chair', 'barstool', 'pepper spray', 'spray', 'wasp spray', 'piece of wood',
        'toy weapon', 'torch', 'flashlight', 'air conditioner', 'hand torch', "contractor's level", 'chain', 'stapler'],
    'Hand tools':
        ['metal hand tool'],
    'Vehicles':
        ['motorcycle', 'car', 'van', 'wagon', 'bike', 'vehicle'],
    'Electrical devices':
        ['taser'],
    'Explosives':
        ['fireworks', 'grenade', 'molotov cocktail', 'incendiary device']
}
unknowns_dict = ['unknown', 'undetermined', 'unidentifiable', 'claimed to be armed']

In [20]:
shootings = pd.read_csv('D:/Datasets/Police Shootings/police shootings/shootings.csv', index_col = 'id')
shootings.columns

Index(['name', 'date', 'manner_of_death', 'armed', 'age', 'gender', 'race',
       'city', 'state', 'signs_of_mental_illness', 'threat_level', 'flee',
       'body_camera'],
      dtype='object')

In [4]:
(round(shootings.isna().sum()/shootings.shape[0]*100, 2)).astype('str') + '%'

name                        0.0%
date                        0.0%
manner_of_death             0.0%
armed                      4.19%
age                        4.34%
gender                     0.04%
race                       9.62%
city                        0.0%
state                       0.0%
signs_of_mental_illness     0.0%
threat_level                0.0%
flee                        4.6%
body_camera                 0.0%
dtype: object

In [5]:
shootings.age.loc[shootings.age.isnull()] = shootings.age.mean()

F:\conda\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
shootings.armed.loc[shootings.armed.isnull()] = 'Unknown'

In [7]:
shootings.gender.loc[shootings.gender.isnull()] = 'M'

In [8]:
shootings.flee.loc[shootings.flee.isnull()] = 'Other'

In [9]:
shootings.manner_of_death.value_counts()

shot                5146
shot and Tasered     270
Name: manner_of_death, dtype: int64

In [10]:
shootings.armed.unique().size

94

In [11]:
shootings = shootings.loc[shootings.race.notna()]

In [12]:
def assign_Race(race):
    if race == 'A':
        race = 'Asian'
    elif race == 'B':
        race = 'Black'
    elif race == 'H':
        race = 'Hispanic'
    elif race == 'N':
        race = 'Native'
    elif race == 'O':
        race = 'Other'
    elif race == 'W':
        race = 'White'
    return race
shootings.loc[:,'race'] = shootings.race.apply(assign_Race)

F:\conda\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [13]:
shootings.armed.loc[shootings.armed.str.contains('|'.join(unknowns_dict),case = False)] = 'unknown'

F:\conda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
pd.set_option('display.max_rows',len(shootings.armed.value_counts()))
shootings.armed.unique()

array(['gun', 'unarmed', 'toy weapon', 'nail gun', 'knife', 'unknown',
       'shovel', 'hammer', 'hatchet', 'sword', 'machete', 'box cutter',
       'metal object', 'screwdriver', 'lawn mower blade', 'flagpole',
       'guns and explosives', 'cordless drill', 'metal pole', 'Taser',
       'metal pipe', 'metal hand tool', 'blunt object', 'metal stick',
       'sharp object', 'meat cleaver', 'carjack', 'chain',
       "contractor's level", 'stapler', 'crossbow', 'bean-bag gun',
       'baseball bat and fireplace poker', 'straight edge razor',
       'gun and knife', 'ax', 'brick', 'baseball bat', 'hand torch',
       'chain saw', 'garden tool', 'scissors', 'pole', 'pick-axe',
       'flashlight', 'vehicle', 'spear', 'chair', 'pitchfork',
       'hatchet and gun', 'rock', 'piece of wood', 'bayonet', 'pipe',
       'glass shard', 'motorcycle', 'pepper spray', 'metal rake', 'baton',
       'crowbar', 'oar', 'machete and gun', 'air conditioner',
       'pole and knife', 'beer bottle', 'base

In [15]:
def categorize_arms(row):
    armed = row['armed'].lower()
    if 'and' in armed.split(' '):
        row['arms_category'] = 'Multiple'
    elif armed in dictionary.keys():
        row['arms_category'] = armed
    elif armed == 'unknown':
        row['arms_category'] = 'Unknown'
    elif armed == 'unarmed':
        row['arms_category'] = 'Unarmed'
    else:
        for key, value in dictionary.items():
            if armed in value:
                row['arms_category'] = key
    return row

arms_category = shootings
arms_category['arms_category'] = None
arms_category = arms_category.apply(categorize_arms, axis = 'columns')

F:\conda\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
shootings = arms_category

In [17]:
shootings

,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,arms_category
id,,,,,,,,,,,,,,
3,Tim Elliot,2015-01-02,shot,gun,53.000000,M,Asian,Shelton,WA,True,attack,Not fleeing,False,Guns
4,Lewis Lee Lembke,2015-01-02,shot,gun,47.000000,M,White,Aloha,OR,False,attack,Not fleeing,False,Guns
5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.000000,M,Hispanic,Wichita,KS,False,other,Not fleeing,False,Unarmed
8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.000000,M,White,San Francisco,CA,True,attack,Not fleeing,False,Other unusual objects
9,Michael Rodriguez,2015-01-04,shot,nail gun,39.000000,M,Hispanic,Evans,CO,False,attack,Not fleeing,False,Piercing objects
11,Kenneth Joe Brown,2015-01-04,shot,gun,18.000000,M,White,Guthrie,OK,False,attack,Not fleeing,False,Guns
13,Kenneth Arnold Buck,2015-01-05,shot,gun,22.000000,M,Hispanic,Chandler,AZ,False,attack,Car,False,Guns
15,Brock Nichols,2015-01-06,shot,gun,35.000000,M,White,Assaria,KS,False,attack,Not fleeing,False,Guns
16,Autumn Steele,2015-01-06,shot,unarmed,34.000000,F,White,Burlington,IA,False,other,Not fleeing,True,Unarmed


In [18]:
shootings.to_csv('C:/Users/Ahsen/Desktop/Shooting/shootings.csv')